Week 4 - Social Network Analysis - Jupyter Notebook

## 1. Collect Tweets 
We collect tweets from Twitter API v2. The bearer_token below has a cap, and please only use it for this class.

### 1.1 Search Tweets

In [42]:
import requests
import os
import json
import pandas as pd

# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
#bearer_token = os.environ.get("BEARER_TOKEN")
bearer_token ='Ask Jane For It'

search_url = "https://api.twitter.com/2/tweets/search/recent"

# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
query_params = {'query': 'GenAI OR AI Marketing'
                ,'tweet.fields': 'author_id,created_at,public_metrics'
                ,'user.fields':'username,public_metrics,description,url,verified,verified_type'
                ,'start_time':'2024-02-26T10:14:49Z'
                ,'end_time':'2024-02-27T10:14:49Z'
                ,'expansions':'entities.mentions.username,author_id'
                ,'max_results':10}


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    #print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def main():
    json_response = connect_to_endpoint(search_url, query_params)
    js = json.dumps(json_response, indent=4, sort_keys=True)  # Write it to a file:
    with open('data/search_tweets.json', 'w') as fo: 
        fo.write(js)

    print(js)
 


if __name__ == "__main__":
    main()

{
    "data": [
        {
            "author_id": "1686118425863639041",
            "created_at": "2024-02-27T10:14:43.000Z",
            "edit_history_tweet_ids": [
                "1762420993434210644"
            ],
            "entities": {
                "mentions": [
                    {
                        "end": 11,
                        "id": "1663269489515872267",
                        "start": 0,
                        "username": "GenAi_offi"
                    }
                ]
            },
            "id": "1762420993434210644",
            "public_metrics": {
                "bookmark_count": 0,
                "impression_count": 0,
                "like_count": 0,
                "quote_count": 0,
                "reply_count": 0,
                "retweet_count": 0
            },
            "text": "@GenAi_offi 14JxTP6RxQWMQQzfD7urPTrnnqBbGkaFPpzbR11VUX8p"
        },
        {
            "author_id": "28119151",
            "created_at": "2024-02-2

### 1.2 Read the Data 

In [43]:
# Python program to read
# json file
 
import json
 
# Opening JSON file
f = open('data/search_tweets.json')
 
# returns JSON object as 
# a dictionary
data = json.load(f)
 
# Iterating through the json
# list
for i in data['data']:  #a for loop would loop over every element of the list data['data'].
    print(i)

for i in data['includes']['users']:
    print(i)
 
# Closing file
f.close()

{'author_id': '1686118425863639041', 'created_at': '2024-02-27T10:14:43.000Z', 'edit_history_tweet_ids': ['1762420993434210644'], 'entities': {'mentions': [{'end': 11, 'id': '1663269489515872267', 'start': 0, 'username': 'GenAi_offi'}]}, 'id': '1762420993434210644', 'public_metrics': {'bookmark_count': 0, 'impression_count': 0, 'like_count': 0, 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0}, 'text': '@GenAi_offi 14JxTP6RxQWMQQzfD7urPTrnnqBbGkaFPpzbR11VUX8p'}
{'author_id': '28119151', 'created_at': '2024-02-27T10:14:33.000Z', 'edit_history_tweet_ids': ['1762420949251695046'], 'id': '1762420949251695046', 'public_metrics': {'bookmark_count': 0, 'impression_count': 11, 'like_count': 0, 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0}, 'text': '5 vital questions to ask yourself before using AI at work https://t.co/mMyvlDLrLk #ai #artificialintelligence #genai'}
{'author_id': '1686118425863639041', 'created_at': '2024-02-27T10:14:20.000Z', 'edit_history_tweet_ids': ['1762420

## 2. Construct a Directed Network

Now, we will use the data above to construct a mentioning network.

### 2.1 Remove tweets that do not contain mentions.

In [50]:
# Filter the list to only include dictionaries that have the specified key
filtered_tweets = []
for tweet in data['data']:
    if 'entities' in tweet:
        filtered_tweets.append(tweet)

In [51]:
# The code above can be written as:
#filtered_tweets = [tweet for tweet in data['data'] if 'entities' in tweet]

In [111]:
# add username to the tweet. The tweet dictionary only included author_id but not username.
id_username={} # this is an empty dictionary
for user in data['includes']['users']:
    id_username.update({user['id']: user['username']}) # add key-value pairs to this dictionary.

 
for tweet in filtered_tweets:
    #if tweet['id'] in id_username:
    tweet['username'] = id_username[str(tweet['author_id'])]
 

### 2.2 Generate Edges 
Each dictionary represents a node pare of author_id and username.

In [114]:
edge=[] # This is an empty list.

i=0
for tweet in filtered_tweets:
    source=tweet['username']
    i=i+1
    print("This is the {}th user!".format(i))
    j=1
    for mention in tweet['entities']['mentions']:
            pair={"source":source,"target":mention["username"]}
            edge.append(pair)
            print("This is the {}th mentioned user!".format(j))
            j=j+1   

This is the 1th user!
This is the 1th mentioned user!
This is the 2th user!
This is the 1th mentioned user!
This is the 3th user!
This is the 1th mentioned user!
This is the 4th user!
This is the 1th mentioned user!
This is the 5th user!
This is the 1th mentioned user!
This is the 2th mentioned user!
This is the 6th user!
This is the 1th mentioned user!
This is the 2th mentioned user!
This is the 7th user!
This is the 1th mentioned user!
This is the 2th mentioned user!
[{'source': 'SolomonChu69387', 'target': 'GenAi_offi'}, {'source': 'SolomonChu69387', 'target': 'GenAi_offi'}, {'source': 'MathurAnoop', 'target': 'cxocapital'}, {'source': 'usergrowth', 'target': 'Streetwisejnl'}, {'source': 'LordMogorCrypto', 'target': 'bullishbestie'}, {'source': 'LordMogorCrypto', 'target': 'CryptoThro'}, {'source': 'cihanev49349281', 'target': 'bullishbestie'}, {'source': 'cihanev49349281', 'target': 'CryptoThro'}, {'source': 'WSBSBULL', 'target': 'bullishbestie'}, {'source': 'WSBSBULL', 'target': '

In [170]:
edge_df=pd.DataFrame(edge)

In [129]:
edge_df

source         target
0  SolomonChu69387     GenAi_offi
1  SolomonChu69387     GenAi_offi
2      MathurAnoop     cxocapital
3       usergrowth  Streetwisejnl
4  LordMogorCrypto  bullishbestie
5  LordMogorCrypto     CryptoThro
6  cihanev49349281  bullishbestie
7  cihanev49349281     CryptoThro
8         WSBSBULL  bullishbestie
9         WSBSBULL     CryptoThro

### 2.3 Generate the Network

In [181]:
G=nx.from_pandas_edgelist(edge_df,
                            source="source",
                            target="target",
                            create_using=nx.DiGraph()) # No need to add create_using=nx.DiGraph() if it is an undirected graph

 
nx.draw(G, with_labels=True, node_color='skyblue', edge_color='gray',pos = nx.circular_layout(G))


## 3. Node Attributes
Before, we've been using the data we collected on the fly during class time. Due to Twitter's API limit, we won't be able to collect too many tweets. Below, we will start to use a dataset I have collected before class. Please stick to this dataset for further analysis.

- edgelist.csv
- user_info.csv

In [204]:
edge_df=pd.read_csv("data/edgelist.csv")

G=nx.from_pandas_edgelist(edge_df,
                            source="source",
                            target="target",
                            create_using=nx.DiGraph()) # No need to add create_using=nx.DiGraph() if it is an undirected graph

nx.draw(G, with_labels=False, node_size=5,node_color='skyblue', edge_color='gray',pos= nx.circular_layout(G))


### 3.1 Centrality

In [205]:
# Calculate eigenvector centrality
eigenvector_centrality = nx.eigenvector_centrality(G)
# Calculate in-degree centrality
in_degree_centrality = nx.in_degree_centrality(G)
# Calculate out-degree centrality
out_degree_centrality = nx.out_degree_centrality(G)
# Calculate betweenness centrality
betweenness_centrality = nx.betweenness_centrality(G)
# Calculate closeness centrality
closeness_centrality = nx.closeness_centrality(G)

### 3.2 Save Centrality as Node Attributes

In [206]:
nx.set_node_attributes(G, eigenvector_centrality, 'eigenvector_centrality')
nx.set_node_attributes(G, in_degree_centrality, 'in_degree_centrality')
nx.set_node_attributes(G, out_degree_centrality, 'out_degree_centrality')
nx.set_node_attributes(G, betweenness_centrality, 'betweenness_centrality')
nx.set_node_attributes(G, closeness_centrality, 'closeness_centrality')

### 3.3 Examine Node Attributes 

In [207]:
for node, attributes in G.nodes(data=True):
    print(node)
    print(attributes)

gladeolie
{'eigenvector_centrality': 9.510434606460809e-17, 'in_degree_centrality': 0.0, 'out_degree_centrality': 0.0049382716049382715, 'betweenness_centrality': 0.0, 'closeness_centrality': 0.0}
Infosys
{'eigenvector_centrality': 7.323034646974816e-15, 'in_degree_centrality': 0.0049382716049382715, 'out_degree_centrality': 0.0, 'betweenness_centrality': 0.0, 'closeness_centrality': 0.0049382716049382715}
RafaelNadal
{'eigenvector_centrality': 7.323034646974816e-15, 'in_degree_centrality': 0.0049382716049382715, 'out_degree_centrality': 0.0, 'betweenness_centrality': 0.0, 'closeness_centrality': 0.0049382716049382715}
onr273
{'eigenvector_centrality': 9.510434606460809e-17, 'in_degree_centrality': 0.0, 'out_degree_centrality': 0.0024691358024691358, 'betweenness_centrality': 0.0, 'closeness_centrality': 0.0}
BrnMetaverse
{'eigenvector_centrality': 7.323034646974816e-15, 'in_degree_centrality': 0.0049382716049382715, 'out_degree_centrality': 0.0, 'betweenness_centrality': 0.0, 'closene

### 3.4 External Attributes

In [212]:
user_attribute=pd.read_csv("data/users_info.csv")
user_attribute.columns

Index(['protected', 'username', 'name', 'url', 'verified', 'location', 'id',
       'description', 'created_at', 'public_metrics.followers_count',
       'public_metrics.following_count', 'public_metrics.tweet_count',
       'public_metrics.listed_count', 'public_metrics.like_count',
       'withheld.country_codes'],
      dtype='object')

In [213]:
for index, row in user_attribute.iterrows():
    node_id = row['username']
    attributes = row.drop('username').to_dict()  # Convert row to dictionary excluding node_id
    nx.set_node_attributes(G, {node_id: attributes})

In [214]:
attributes_list=[]
for node, attributes in G.nodes(data=True):
    print(node)
    print(attributes)
    attributes_list.append(attributes)

gladeolie
{'eigenvector_centrality': 9.510434606460809e-17, 'in_degree_centrality': 0.0, 'out_degree_centrality': 0.0049382716049382715, 'betweenness_centrality': 0.0, 'closeness_centrality': 0.0, 'protected': False, 'name': 'ARYAN®', 'url': 'https://t.co/1PaUcEKvJT', 'verified': False, 'location': 'cyberspace', 'id': 54516759, 'description': 'Just a common man with a scientific temper.\n#geek #technosavvy #geopolitics #nature #health #astrophyscics #blogger #fashion\n\nRetweet is not an endorsement.®©', 'created_at': '2009-07-07T11:18:59.000Z', 'public_metrics.followers_count': 1541, 'public_metrics.following_count': 2645, 'public_metrics.tweet_count': 423931, 'public_metrics.listed_count': 227, 'public_metrics.like_count': 25291, 'withheld.country_codes': nan}
Infosys
{'eigenvector_centrality': 7.323034646974816e-15, 'in_degree_centrality': 0.0049382716049382715, 'out_degree_centrality': 0.0, 'betweenness_centrality': 0.0, 'closeness_centrality': 0.0049382716049382715, 'protected': F

## 4. Predictive Analysis

### 4.1 Variable Conversion

In [215]:
# verified is logical and needs to be converted to numeric before being used as a predictor.
attributes_df=pd.DataFrame(attributes_list)
attributes_df['verified']=attributes_df['verified'].astype(int)

In [216]:
attributes_df.columns

Index(['eigenvector_centrality', 'in_degree_centrality',
       'out_degree_centrality', 'betweenness_centrality',
       'closeness_centrality', 'protected', 'name', 'url', 'verified',
       'location', 'id', 'description', 'created_at',
       'public_metrics.followers_count', 'public_metrics.following_count',
       'public_metrics.tweet_count', 'public_metrics.listed_count',
       'public_metrics.like_count', 'withheld.country_codes'],
      dtype='object')

### 4.2 Assembling the Data

In [217]:
# Predictors
X=attributes_df[['eigenvector_centrality','public_metrics.like_count','public_metrics.following_count','public_metrics.tweet_count', 'verified','in_degree_centrality', 'out_degree_centrality',
       'betweenness_centrality', 'closeness_centrality']]
# Outcome
y=np.log1p(attributes_df['public_metrics.followers_count'].astype(int) )

### 4.3 Training and Testing Partition

In [218]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


### 4.4 Model Fitting

In [166]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

 # Initialize the RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
regressor.fit(X, y)
# Make predictions and evaluate the model

 

RandomForestRegressor(random_state=42)

### 4.5 Validation

In [167]:
predictions = regressor.predict(X_test)
# Evaluate the model
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)

print("Mean Squared Error: {}".format(mse))
print("Root Mean Squared Error: {}".format(rmse))
 

Mean Squared Error: 0.6249943324626919
Root Mean Squared Error: 0.7905658305686453


In [168]:
from matplotlib import pyplot as plt
#feature_names = [f"feature {i}" for i in range(W.shape[1])]
global_importances = pd.Series(regressor.feature_importances_, index=X.columns)
global_importances.sort_values(ascending=True, inplace=True)
global_importances.plot.barh(color='green')
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.title("Feature Importance - Follower")
global_importances

betweenness_centrality            0.003387
eigenvector_centrality            0.005765
out_degree_centrality             0.038092
in_degree_centrality              0.064893
public_metrics.like_count         0.074521
verified                          0.084986
public_metrics.following_count    0.160062
public_metrics.tweet_count        0.267792
closeness_centrality              0.300502
dtype: float64